# Sparsifying ResNet-50 from Scratch (Beans)

In this example, we will demonstrate how to sparsify an image classification model from scratch using SparseML's PyTorch integration. We train and prune [ResNet-50](https://pytorch.org/vision/main/models/generated/torchvision.models.resnet50.html) on the downstream [Beans dataset](https://huggingface.co/datasets/beans) using the Global Magnitude Pruning algorithm. 

## Agenda

There are a few steps:

 1. Setup the dataset
 2. Setup the PyTorch training loop
 3. Train a dense version of ResNet-50
 4. Run the GMP pruning algorithm on the dense model
 
## Installation

Install SparseML with `pip`:

```
pip install sparseml[torchvision]
```

In [1]:
import torch
import sparseml
import torchvision
from sparseml.pytorch.optim import ScheduledModifierManager
from sparseml.pytorch.utils import TensorBoardLogger, ModuleExporter, get_prunable_layers, tensor_sparsity
from torch.utils.data import DataLoader
from torch.nn import CrossEntropyLoss
from torch.optim import Adam
from torchvision import transforms
from tqdm.auto import tqdm
import math
import datasets

## Step 1: Setup Dataset

Beans leaf dataset is a set of images of diseased and healthy leaves. Based on a leaf image, the goal of this task is to predict the disease type (Angular Leaf Spot and Bean Rust), if any.

We will use the Hugging Face `datasets` library to download the data and the torchvision `ImageFolder` in the training loop.

[Checkout the dataset card](https://huggingface.co/datasets/beans)

In [ ]:
beans_dataset = datasets.load_dataset("beans")

In [3]:
print(beans_dataset["train"][0]["image_file_path"])
print(beans_dataset["validation"][0]["image_file_path"])

/home/ubuntu/.cache/huggingface/datasets/downloads/extracted/eeb026374cf5ecfd5f40131a3159be9b9055ac21a3da11690e7eb4d117c99eee/train/bean_rust/bean_rust_train.84.jpg
/home/ubuntu/.cache/huggingface/datasets/downloads/extracted/f287261265d2f9a3e8f87a5526a54d1847b17f7c3ec5714e5719432f2b3e4a73/validation/bean_rust/bean_rust_val.36.jpg


In [4]:
train_path = "/home/ubuntu/.cache/huggingface/datasets/downloads/extracted/eeb026374cf5ecfd5f40131a3159be9b9055ac21a3da11690e7eb4d117c99eee/train"
val_path = "/home/ubuntu/.cache/huggingface/datasets/downloads/extracted/f287261265d2f9a3e8f87a5526a54d1847b17f7c3ec5714e5719432f2b3e4a73/validation"

In [5]:
NUM_LABELS = 3
BATCH_SIZE = 64

# imagenet transforms
imagenet_transform = transforms.Compose([
   transforms.Resize(size=256, interpolation=transforms.InterpolationMode.BILINEAR, max_size=None, antialias=None),
   transforms.CenterCrop(size=(224, 224)),
   transforms.ToTensor(),
   transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

# datasets
train_dataset = torchvision.datasets.ImageFolder(
    root=train_path,
    transform=imagenet_transform
)

val_dataset = torchvision.datasets.ImageFolder(
    root=val_path,
    transform=imagenet_transform
)


# dataloaders
train_loader = DataLoader(train_dataset, BATCH_SIZE, shuffle=True, pin_memory=True, num_workers=16)
val_loader = DataLoader(val_dataset, BATCH_SIZE, shuffle=False, pin_memory=True, num_workers=16)

## Step 2: Setup PyTorch Training Loop

We will use this training loop below. This is standard PyTorch functionality.

In [6]:
device = "cuda" if torch.cuda.is_available() else "cpu"
print(device)

def run_model_one_epoch(model, data_loader, criterion, device, train=False, optimizer=None):
    if train:
        model.train()
    else:
        model.eval()

    running_loss = 0.0
    total_correct = 0
    total_predictions = 0

    # loop through batches
    for step, (inputs, labels) in tqdm(enumerate(data_loader), total=len(data_loader)):
        inputs = inputs.to(device)
        labels = labels.to(device)

        if train:
            optimizer.zero_grad()

        # compute loss, run backpropogation
        outputs = model(inputs)  # model returns logits
        loss = criterion(outputs, labels)
        if train:
            loss.backward()
            optimizer.step()

        running_loss += loss.item()

        # run evaluation
        predictions = outputs.argmax(dim=1)
        total_correct += torch.sum(predictions == labels).item()
        total_predictions += inputs.size(0)

    # return loss and evaluation metric
    loss = running_loss / (step + 1.0)
    accuracy = total_correct / total_predictions
    return loss, accuracy

cuda


## **Step 3: Train ResNet-50 on Beans**

First, we will train a dense version of ResNet-50 on the Beans dataset.

In [7]:
# download pre-trained model, setup classification head
model = torchvision.models.resnet50(weights=torchvision.models.ResNet50_Weights.DEFAULT)
model.fc = torch.nn.Linear(model.fc.in_features, NUM_LABELS)
model.to(device)

# setup loss function and optimizer
criterion = CrossEntropyLoss()
optimizer = Adam(model.parameters(), lr=8e-3) # lr will be override by sparseml

Next, we will use SparseML's recipes to set the hyperparameters of training loop. In this case, we will use the following recipe:

```yaml
# Epoch and Learning-Rate variables
num_epochs: 10.0
init_lr: 0.0005

training_modifiers:
  - !EpochRangeModifier
    start_epoch: 0.0
    end_epoch: eval(num_epochs)

  - !LearningRateFunctionModifier
    final_lr: 0.0
    init_lr: eval(init_lr)
    lr_func: cosine
    start_epoch: 0.0
    end_epoch: eval(num_epochs)
```

As you can see, the recipe includes an `!EpochRangeModifier` and a `!LearningRateFunctionModifier`. These modifiers simply set the number of epochs to train for and the learning rate schedule. As a result, the final model will be dense.

In [ ]:
!cat ./recipes/resnet50-beans-dense-recipe.yaml

In [9]:
dense_recipe_path = "./recipes/resnet50-beans-dense-recipe.yaml"

Next, we use SparseML's `ScheduledModifierManager` to parse and apply the recipe. The `manager.modify` function modifies and wraps the `model` and `optimizer` with the instructions from the recipe. You can use the `model` and `optimizer` just like standard PyTorch objects.

In [10]:
# create ScheduledModifierManager and Optimizer wrapper
manager = ScheduledModifierManager.from_yaml(dense_recipe_path)
optimizer = manager.modify(model, optimizer, steps_per_epoch=len(train_loader))

Kick off the transfer learning loop. Our run reached ~99% validation accuracy after 10 epochs.

In [11]:
# run transfer learning
epoch = 0
for epoch in range(manager.max_epochs):
    # run training loop
    epoch_name = f"{epoch + 1}/{manager.max_epochs}"
    print(f"Running Training Epoch {epoch_name}")
    train_loss, train_acc = run_model_one_epoch(model, train_loader, criterion, device, train=True, optimizer=optimizer)
    print(f"Training Epoch: {epoch_name}\nTraining Loss: {train_loss}\nTop 1 Acc: {train_acc}\n")

    # run validation loop
    print(f"Running Validation Epoch {epoch_name}")
    val_loss, val_acc = run_model_one_epoch(model, val_loader, criterion, device)
    print(f"Validation Epoch: {epoch_name}\nVal Loss: {val_loss}\nTop 1 Acc: {val_acc}\n")

manager.finalize(model)

Running Training Epoch 1/10


  0%|          | 0/17 [00:00<?, ?it/s]

Training Epoch: 1/10
Training Loss: 0.4227903384496184
Top 1 Acc: 0.8355899419729207

Running Validation Epoch 1/10


  0%|          | 0/3 [00:00<?, ?it/s]

Validation Epoch: 1/10
Val Loss: 0.166859846872588
Top 1 Acc: 0.9398496240601504

Running Training Epoch 2/10


...

Validation Epoch: 9/10
Val Loss: 0.024938953380721312
Top 1 Acc: 0.9849624060150376

Running Training Epoch 10/10


  0%|          | 0/17 [00:00<?, ?it/s]

Training Epoch: 10/10
Training Loss: 0.0028374234782805774
Top 1 Acc: 1.0

Running Validation Epoch 10/10


  0%|          | 0/3 [00:00<?, ?it/s]

Validation Epoch: 10/10
Val Loss: 0.021825105145884056
Top 1 Acc: 0.9924812030075187



In [14]:
from sparseml.pytorch.utils import ModuleExporter

save_dir = "dense_model"
exporter = ModuleExporter(model, output_dir=save_dir)
exporter.export_pytorch(name="resnet-50-dense-beans.pth")
exporter.export_onnx(torch.randn(1, 3, 224, 224), name="dense-model.onnx", convert_qat=True)

In [15]:
torch.cuda.empty_cache()

## Step 4: Prune The Model

With a model trained on Beans, we are now ready to apply the GMP algorithm to prune the model. The GMP algorithm is an interative pruning algorithm. At the end of each epoch, we identify the lowest magnitude weights (those closest to 0) and remove them from the network starting from an initial level of sparsity until a final level of sparsity. The remaining nonzero weights are then fine-tuned onto training dataset.

In [16]:
checkpoint = torch.load("./dense_model/training/resnet-50-dense-beans.pth")
model = torchvision.models.resnet50()
model.fc = torch.nn.Linear(model.fc.in_features, NUM_LABELS)
model.load_state_dict(checkpoint['state_dict'])
model.to(device)

# setup loss function and optimizer, LR will be overriden by sparseml
criterion = CrossEntropyLoss()
optimizer = Adam(model.parameters(), lr=8e-3)

Next, we need to create a SparseML recipe which includes the GMP algorithm. The `!GlobalMagnitudePruningModifier` modifier instructs SparseML to apply the GMP algorithm at a global level (pruning the lowest magnitude weights across all layers).

Firstly, we need to decide identify which parameters of the model to apply the GMP algorithm to. We can use the `get_prunable_layers` function to inspect:

In [17]:
# print parameters
for (name, layer) in get_prunable_layers(model):
    print(f"{name}")

conv1
layer1.0.conv1
layer1.0.conv2
layer1.0.conv3
layer1.0.downsample.0
layer1.1.conv1
layer1.1.conv2
layer1.1.conv3
layer1.2.conv1
layer1.2.conv2
layer1.2.conv3
layer2.0.conv1
layer2.0.conv2
layer2.0.conv3
layer2.0.downsample.0
layer2.1.conv1
layer2.1.conv2
layer2.1.conv3
layer2.2.conv1
layer2.2.conv2
layer2.2.conv3
layer2.3.conv1
layer2.3.conv2
layer2.3.conv3
layer3.0.conv1
layer3.0.conv2
layer3.0.conv3
layer3.0.downsample.0
layer3.1.conv1
layer3.1.conv2
layer3.1.conv3
layer3.2.conv1
layer3.2.conv2
layer3.2.conv3
layer3.3.conv1
layer3.3.conv2
layer3.3.conv3
layer3.4.conv1
layer3.4.conv2
layer3.4.conv3
layer3.5.conv1
layer3.5.conv2
layer3.5.conv3
layer4.0.conv1
layer4.0.conv2
layer4.0.conv3
layer4.0.downsample.0
layer4.1.conv1
layer4.1.conv2
layer4.1.conv3
layer4.2.conv1
layer4.2.conv2
layer4.2.conv3
fc


We will apply pruning to each of the convs and exclude the fc layer (which is the final projection head). Fortunately, SparseML allows us to pass regexes to identify layers in the network, so we can use the following list to identify the relevant layers for pruning:
    
    - 'conv1.weight'
    - 're:layer1.*.conv1.weight'
    - 're:layer1.*.conv2.weight'
    - 're:layer1.*.conv3.weight'
    - 're:layer1.0.downsample.0.weight'
    - 're:layer2.*.conv1.weight'
    - 're:layer2.*.conv2.weight'
    - 're:layer2.*.conv3.weight'
    - 're:layer2.0.downsample.0.weight'
    - 're:layer3.*.conv1.weight'
    - 're:layer3.*.conv2.weight'
    - 're:layer3.*.conv3.weight'
    - 're:layer3.0.downsample.0.weight'
    - 're:layer4.*.conv1.weight'
    - 're:layer4.*.conv2.weight'
    - 're:layer4.*.conv3.weight'
    - 're:layer4.0.downsample.0.weight'

Here is what the recipe looks like:

```yaml
# Epoch and Learning-Rate variables
num_epochs: 13.0
pruning_epochs: 10.0
init_lr: 0.00025
inter_func: cubic
mask_type: unstructured

training_modifiers:
  - !EpochRangeModifier
    start_epoch: 0.0
    end_epoch: eval(num_epochs)

  - !LearningRateFunctionModifier
    final_lr: 0.0
    init_lr: eval(init_lr)
    lr_func: cosine
    start_epoch: 0.0
    end_epoch: eval(num_epochs)

# Pruning
pruning_modifiers:
   - !GlobalMagnitudePruningModifier
     init_sparsity: 0.05
     final_sparsity: 0.90
     start_epoch: 0.0
     end_epoch: eval(pruning_epochs)
     update_frequency: 1.0
     params: 
        - 'conv1.weight'
        - 're:layer1.*.conv1.weight'
        - 're:layer1.*.conv2.weight'
        - 're:layer1.*.conv3.weight'
        - 're:layer1.0.downsample.0.weight'
        - 're:layer2.*.conv1.weight'
        - 're:layer2.*.conv2.weight'
        - 're:layer2.*.conv3.weight'
        - 're:layer2.0.downsample.0.weight'
        - 're:layer3.*.conv1.weight'
        - 're:layer3.*.conv2.weight'
        - 're:layer3.*.conv3.weight'
        - 're:layer3.0.downsample.0.weight'
        - 're:layer4.*.conv1.weight'
        - 're:layer4.*.conv2.weight'
        - 're:layer4.*.conv3.weight'
        - 're:layer4.0.downsample.0.weight'
     leave_enabled: True
     inter_func: eval(inter_func)
     mask_type: eval(mask_type)
```

This recipe specifies that we will run the GMP algorithm for the first 10 epochs. We start at an init_sparsity level of 5% and gradually increase sparsity to a final_sparsity level of 90% following a cubic curve. The pruning is applied in an unstructured manner, meaning that any weight can be pruned.

Over the final 3 epochs, we will fine-tune the 90% pruned model further. Since we set leave_enabled=True the sparsity level will be maintained as the fine-tuning occurs.

In [ ]:
!cat recipes/resnet50-beans-pruning-recipe.yaml

In [19]:
pruning_recipe_path = "./recipes/resnet50-beans-pruning-recipe.yaml"

In [20]:
# create ScheduledModifierManager and Optimizer wrapper
manager = ScheduledModifierManager.from_yaml(pruning_recipe_path)
logger = TensorBoardLogger(log_path="./tensorboard_outputs")
optimizer = manager.modify(model, optimizer, loggers=[logger], steps_per_epoch=len(train_loader))

Next, kick off the GMP training loop. 

As you can see, we use the wrapped `optimizer` and `model` in the same way as above. SparseML parsed the recipe and updated the `optimizer` with the logic of GMP algorithm from the recipe. This allows you to just the `optimizer` and `model` as usual, with all of the pruning-related logic specified by the declarative recipe interface.

Our 90% pruned model reaches ~99% validation accuracy (vs ~99% for the dense model).

In [21]:
# run GMP algorithm
epoch = 0
for epoch in range(manager.max_epochs):
    # run training loop
    epoch_name = f"{epoch + 1}/{manager.max_epochs}"
    print(f"Running Training Epoch {epoch_name}")
    train_loss, train_acc = run_model_one_epoch(model, train_loader, criterion, device, train=True, optimizer=optimizer)
    print(f"Training Epoch: {epoch_name}\nTraining Loss: {train_loss}\nTop 1 Acc: {train_acc}\n")

    # run validation loop
    print(f"Running Validation Epoch {epoch_name}")
    val_loss, val_acc = run_model_one_epoch(model, val_loader, criterion, device)
    print(f"Validation Epoch: {epoch_name}\nVal Loss: {val_loss}\nTop 1 Acc: {val_acc}\n")
    
    logger.log_scalar("Metrics/Loss (Train)", train_loss, epoch)
    logger.log_scalar("Metrics/Accuracy (Train)", train_acc, epoch)
    logger.log_scalar("Metrics/Loss (Validation)", val_loss, epoch)
    logger.log_scalar("Metrics/Accuracy (Validation)", val_acc, epoch)
    
manager.finalize(model)

Running Training Epoch 1/13


  0%|          | 0/17 [00:00<?, ?it/s]

Training Epoch: 1/13
Training Loss: 0.016460627817329678
Top 1 Acc: 0.995164410058027

Running Validation Epoch 1/13


  0%|          | 0/3 [00:00<?, ?it/s]

Validation Epoch: 1/13
Val Loss: 0.025849940720945597
Top 1 Acc: 0.9849624060150376

Running Training Epoch 2/13


...

Validation Epoch: 12/13
Val Loss: 0.013252861564978957
Top 1 Acc: 1.0

Running Training Epoch 13/13


  0%|          | 0/17 [00:00<?, ?it/s]

Training Epoch: 13/13
Training Loss: 0.005194885152912534
Top 1 Acc: 1.0

Running Validation Epoch 13/13


  0%|          | 0/3 [00:00<?, ?it/s]

Validation Epoch: 13/13
Val Loss: 0.012128440197557211
Top 1 Acc: 1.0



Here is a sample of the TensorBoard output, showing the validation accuracy, a particular layer's sparsity level, and the learning rate over time.

![tensorboard output](./images/resnet-beans-tensorboard-output.png)

We can print layer-by-layer sparsity as well.

In [22]:
for (name, layer) in get_prunable_layers(model):
    print(f"{name}.weight: {tensor_sparsity(layer.weight).item():.4f}")

conv1.weight: 0.4039
layer1.0.conv1.weight: 0.4758
layer1.0.conv2.weight: 0.6846
layer1.0.conv3.weight: 0.5506
layer1.0.downsample.0.weight: 0.5157
layer1.1.conv1.weight: 0.5690
layer1.1.conv2.weight: 0.6410
layer1.1.conv3.weight: 0.5604
layer1.2.conv1.weight: 0.5430
layer1.2.conv2.weight: 0.6075
layer1.2.conv3.weight: 0.6064
layer2.0.conv1.weight: 0.5498
layer2.0.conv2.weight: 0.7556
layer2.0.conv3.weight: 0.6881
layer2.0.downsample.0.weight: 0.8303
layer2.1.conv1.weight: 0.7918
layer2.1.conv2.weight: 0.8089
layer2.1.conv3.weight: 0.7450
layer2.2.conv1.weight: 0.7564
layer2.2.conv2.weight: 0.8100
layer2.2.conv3.weight: 0.7706
layer2.3.conv1.weight: 0.7239
layer2.3.conv2.weight: 0.7862
layer2.3.conv3.weight: 0.7415
layer3.0.conv1.weight: 0.6814
layer3.0.conv2.weight: 0.8697
layer3.0.conv3.weight: 0.7558
layer3.0.downsample.0.weight: 0.9209
layer3.1.conv1.weight: 0.8875
layer3.1.conv2.weight: 0.9177
layer3.1.conv3.weight: 0.8482
layer3.2.conv1.weight: 0.9054
layer3.2.conv2.weight: 0.914

Finally, export your model to ONNX.

In [23]:
save_dir = "experiment-0"
exporter = ModuleExporter(model, output_dir=save_dir)
exporter.export_pytorch(name="resnet-50-sparse-beans.pth")
exporter.export_onnx(torch.randn(1, 3, 224, 224), name="sparse-model.onnx", convert_qat=True)

## Wrap Up

The resulting model is is 90% sparse and achieves validation accuracy of ~100% (vs the unoptimized dense model at ~99%) without much hyperparameter search.

Key hyperparameter experiments you may want to run include:
- Learning rate
- Learning rate schedule
- Sparsity level
- Number of pruning epochs

DeepSparse supports speedup from pruning and quantization. To reach maximum performance, check out our examples of quantizing a model!